In [2]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [3]:
"""
Experiment: 2026_02_07_exp_031_final_custom_cnn_data_medianblur_224
Goal:
Dataset:
Notes:
"""

'\nExperiment: 2026_02_07_exp_031_final_custom_cnn_data_medianblur_224\nGoal:\nDataset:\nNotes:\n'

In [4]:
import os
os.chdir("/content/drive/My Drive/Colab Notebooks/Data Science Group Project")

print(os.getcwd())
print(os.listdir())

/content/drive/My Drive/Colab Notebooks/Data Science Group Project
['data', 'experiments', 'JustTests', 'OLD', 'MoveImagesFinal.ipynb', 'Splitting Dataset Into Train_Validation_Test_Sets.ipynb', 'RemovingBlackFinal.ipynb', 'Top-View Image Selection From MRI and CT Dataset.ipynb', 'old experiments']


In [5]:
# =====================================================
# Imports
# =====================================================

import platform
from pathlib import Path
import os
import cv2
import torch
import torch.multiprocessing as mp
from torch import nn
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
import albumentations as A
from albumentations.pytorch import ToTensorV2
import numpy as np
import yaml
import json
import pandas as pd
import random

# =====================================================
# Config & Reproducibility
# =====================================================

def load_config(path):
    with open(path, "r") as f:
        return yaml.safe_load(f)


# =====================================================
# Dataset
# =====================================================

class DualImageDataset(Dataset):
    def __init__(self, path):
        # Load once
        self.raw_imgs, self.proc_imgs, self.labels = torch.load(path)

        # Ensure proper dtype
        self.raw_imgs = self.raw_imgs.float()
        self.proc_imgs = self.proc_imgs.float()
        self.labels = self.labels.long()

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return self.raw_imgs[idx], self.proc_imgs[idx], self.labels[idx]

def dataset(transformed_data_dir, cfg):
    train_dataset = DualImageDataset(path=transformed_data_dir / cfg["data"]["train_path"])
    val_dataset = DualImageDataset(path=transformed_data_dir / cfg["data"]["val_path"])
    test_dataset = DualImageDataset(path=transformed_data_dir / cfg["data"]["test_path"])

    return train_dataset, val_dataset, test_dataset


# =====================================================
# Dataloaders
# =====================================================

def dataloader(cfg, train_dataset, val_dataset, test_dataset):
    train_dataloader = DataLoader(train_dataset, batch_size=cfg["training"]["batch_size"], shuffle=True, num_workers=0, pin_memory=True)
    val_dataloader = DataLoader(val_dataset, batch_size=cfg["training"]["batch_size"], shuffle=False, num_workers=0, pin_memory=True)
    test_dataloader = DataLoader(test_dataset, batch_size=cfg["training"]["batch_size"], shuffle=False, num_workers=0, pin_memory=True)

    print(f"Number of training samples: {len(train_dataset)}")
    print(f"Number of validation samples: {len(val_dataset)}")
    print(f"Number of testing samples: {len(test_dataset)}")

    print(f"Length of TrainDataloader: {len(train_dataloader)} batches of {cfg['training']['batch_size']}")
    print(f"Length of ValDataloader: {len(val_dataloader)} batches of {cfg['training']['batch_size']}")
    print(f"Length of TestDataloader: {len(test_dataloader)} batches of {cfg['training']['batch_size']}")

    return train_dataloader, val_dataloader, test_dataloader


# =====================================================
# Model
# =====================================================

class CustomCNN(nn.Module):
    def __init__(self, input_shape, hidden_units, output_shape, dropout, cfg):
        super().__init__()

        self.conv_block_1 = nn.Sequential(
            nn.Conv2d(in_channels=input_shape, out_channels=hidden_units, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=hidden_units, out_channels=hidden_units, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)
        )

        self.conv_block_2 = nn.Sequential(
            nn.Conv2d(in_channels=hidden_units, out_channels=hidden_units, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=hidden_units, out_channels=hidden_units, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)
        )

        # Compute flatten size dynamically
        with torch.no_grad():
            x = torch.zeros(1, input_shape, *tuple(cfg["data"]["image_size"]))  # batch_size=1, input_shape channels
            x = self.conv_block_1(x)
            x = self.conv_block_2(x)
            n_features = x.numel() // x.shape[0]  # total features per sample

        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Dropout(p=dropout),
            nn.Linear(in_features=n_features, out_features=output_shape)
        )

    def forward(self, raw_img, processed_img):
        x = torch.cat([raw_img, processed_img], dim=1)  # Concatenate raw + processed channels -> [B, 2, H, W]
        x = self.conv_block_1(x)
        x = self.conv_block_2(x)
        x = self.classifier(x)
        return x


def make_model(cfg, classes, device):
    model = CustomCNN(input_shape=cfg["model"]["input_dim"], hidden_units=cfg["model"]["hidden_units"],
                      output_shape=len(classes), dropout=cfg["model"]["dropout"], cfg=cfg).to(device)

    loss_func = nn.CrossEntropyLoss()
    optimizer = torch.optim.RMSprop(model.parameters(), lr=cfg["training"]["lr"])
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode="min", factor=cfg["training"]["factor"], patience=cfg["training"]["patience"])

    return model, loss_func, optimizer, scheduler


# =====================================================
# Training / Evaluation Utils
# =====================================================

# =====================================================
# Train
# =====================================================

def train_step(model, train_dataloader, loss_func, optimizer, device):
    train_loss, train_acc = 0, 0
    correct = 0
    total = 0

    model.train()

    for batch, (raw_X, processed_X, y) in enumerate(train_dataloader):
        raw_X, processed_X, y = raw_X.to(device, non_blocking=True), processed_X.to(device, non_blocking=True), y.to(device)

        train_y_pred = model(raw_X, processed_X)

        loss = loss_func(train_y_pred, y)
        train_loss += loss.item()

        # accuracy
        correct += (train_y_pred.argmax(dim=1) == y).sum().item()
        total += y.size(0)

        optimizer.zero_grad()

        loss.backward()

        optimizer.step()

    train_loss /= len(train_dataloader)
    train_acc = 100.0 * correct / total

    print(f"Train loss: {train_loss:.5f} | Train acc: {train_acc:.2f}%\n")
    return train_loss, train_acc


# =====================================================
# Validation
# =====================================================

def val_step(model, val_dataloader, loss_func, device):
    val_loss, val_acc = 0, 0
    correct = 0
    total = 0

    model.eval()
    with torch.inference_mode():
        for raw_X, processed_X, y in val_dataloader:
            raw_X, processed_X, y = raw_X.to(device, non_blocking=True), processed_X.to(device, non_blocking=True), y.to(device)

            val_y_pred = model(raw_X, processed_X)

            val_loss += loss_func(val_y_pred, y).item()

            # accuracy
            correct += (val_y_pred.argmax(dim=1) == y).sum().item()
            total += y.size(0)

        val_loss /= len(val_dataloader)
        val_acc = 100.0 * correct / total

    print(f"Val loss: {val_loss:.5f} | Val acc: {val_acc:.2f}%\n")
    return val_loss, val_acc


def train_and_evaluate(model, epochs, train_dataloader, val_dataloader, loss_func, optimizer, scheduler, device):
    train_loss_list = []
    train_acc_list = []
    val_loss_list = []
    val_acc_list = []

    for epoch in tqdm(range(epochs)):
        train_loss, train_acc = train_step(model, train_dataloader, loss_func, optimizer, device)
        val_loss, val_acc = val_step(model, val_dataloader, loss_func, device)

        scheduler.step(val_loss)

        train_loss_list.append(train_loss)
        train_acc_list.append(train_acc)
        val_loss_list.append(val_loss)
        val_acc_list.append(val_acc)

        current_lr = optimizer.param_groups[0]["lr"]
        print(f"Epoch {epoch+1} | LR: {current_lr:.6e}")

    return train_loss_list, train_acc_list, val_loss_list, val_acc_list


# =====================================================
# Test
# =====================================================

def test_step(model, test_data_loader, loss_func, device):
    test_loss, test_acc = 0, 0
    correct = 0
    total = 0
    y_test_list = []
    y_pred_list = []
    y_pred_prob_list = []

    model.eval()
    with torch.inference_mode():
        for raw_X, processed_X, y in test_data_loader:
            raw_X, processed_X, y = raw_X.to(device, non_blocking=True), processed_X.to(device, non_blocking=True), y.to(device)

            test_y_pred = model(raw_X, processed_X)
            y_pred_prob_list.append(torch.softmax(test_y_pred, dim=1))

            test_loss += loss_func(test_y_pred, y).item()

            y_test_list.append(y.cpu())
            y_pred_list.append(test_y_pred.argmax(dim=1).cpu())

            correct += (test_y_pred.argmax(dim=1) == y).sum().item()
            total += y.size(0)

        test_loss /= len(test_data_loader)
        test_acc = 100.0 * correct / total

    print(f"Test loss: {test_loss:.5f} | Test acc: {test_acc:.2f}%\n")
    return y_test_list, y_pred_list, y_pred_prob_list, test_acc

In [6]:
experiment_path = Path("experiments/2026_02_07_exp_031_final_custom_cnn_data_medianblur_224")

config = load_config(experiment_path / "config.yaml")

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
os.environ["CUBLAS_WORKSPACE_CONFIG"] = ":4096:8"
# torch.use_deterministic_algorithms(True)

random.seed(config["seed"])
np.random.seed(config["seed"])
torch.manual_seed(config["seed"])
torch.cuda.manual_seed(config["seed"])
torch.cuda.manual_seed_all(config["seed"])

device = "cuda" if config["device"] == "cuda" and torch.cuda.is_available() else "cpu"

train_dataset, val_dataset, test_dataset = dataset(Path("data/processed/mri"), config)

train_dataloader, val_dataloader, test_dataloader = dataloader(config, train_dataset, val_dataset, test_dataset)

classes = ['glioma', 'meningioma', 'pituitary']

model, loss_func, optimizer, scheduler = make_model(config, classes, device)

train_loss_list, train_acc_list, val_loss_list, val_acc_list = train_and_evaluate(model,
                                                                                    config["training"]["epochs"],
                                                                                    train_dataloader, val_dataloader,
                                                                                    loss_func, optimizer, scheduler, device)

checkpoint = {
    "epoch": config["training"]["epochs"],
    "model_state_dict": model.state_dict(),
    "optimizer_state_dict": optimizer.state_dict(),
    "config": config,
    "best_val_loss": np.min(val_loss_list),
}

torch.save(checkpoint, experiment_path / "checkpoint.pth")

y_test_list, y_pred_list, y_pred_prob_list, test_accuracy = test_step(model, test_dataloader, loss_func, device)

y_test_list = torch.cat(y_test_list).numpy()  # true labels
y_pred_list = torch.cat(y_pred_list).numpy()  # predicted class
y_pred_prob_list = torch.cat(y_pred_prob_list).cpu().numpy()  # predicted probabilities

print(f"Test: {y_test_list}")
print(f"Predicted: {y_pred_list}")
print(f"Predicted Probability: {y_pred_prob_list}")

# Convert metrics to JSON-friendly format
metrics = {
    "train_loss": [float(l) for l in train_loss_list],
    "train_accuracy": [float(a) for a in train_acc_list],
    "val_loss": [float(l) for l in val_loss_list],
    "val_accuracy": [float(a) for a in val_acc_list],
    "test_accuracy": test_accuracy,
    "classes": classes,
    "epochs": config["training"]["epochs"],
    "num_train_samples": len(train_dataset),
    "num_val_samples": len(val_dataset),
    "num_test_samples": len(test_dataset),
    "best_val_epoch": int(np.argmin(val_loss_list)) + 1,
    "best_val_loss": float(np.min(val_loss_list)),
    "best_val_accuracy": float(val_acc_list[np.argmin(val_loss_list)])
}

# Save metrics
metrics_path = experiment_path / "metrics.json"
with open(metrics_path, "w") as f:
    json.dump(metrics, f, indent=4)

print(f"Metrics saved to {metrics_path}")

# Create DataFrame
metrics_df = pd.DataFrame({
    "epoch": list(range(config["training"]["epochs"])),
    "train_loss": train_loss_list,
    "train_accuracy": [a for a in train_acc_list],
    "val_loss": val_loss_list,
    "val_accuracy": [a for a in val_acc_list]
})

# Save to CSV
metrics_df.to_csv(experiment_path / "train_val_metrics.csv", index=False)
print("Train/Val metrics saved to train_val_metrics.csv")

# Convert predicted probabilities to a DataFrame
prob_df = pd.DataFrame(y_pred_prob_list, columns=[f"prob_{cls}" for cls in classes])

# Create main DataFrame
test_df = pd.DataFrame({"y_true": y_test_list, "y_pred": y_pred_list})

# Concatenate probabilities
test_df = pd.concat([test_df, prob_df], axis=1)

# Save to CSV
test_df.to_csv(experiment_path / "test_predictions.csv", index=False)
print("Test predictions saved to test_predictions.csv")

env_info = {
    "python_version": platform.python_version(),
    "pytorch_version": torch.__version__,
    "cuda_version": torch.version.cuda,
    "cudnn_version": torch.backends.cudnn.version(),
    "gpu": torch.cuda.get_device_name(0) if torch.cuda.is_available() else None,
    "os": platform.platform(),
}

env_path = experiment_path / "env_info.json"
with open(env_path, "w") as f:
    json.dump(env_info, f, indent=4)

print(f"Environment Info saved to {env_path}")

Number of training samples: 3181
Number of validation samples: 908
Number of testing samples: 456
Length of TrainDataloader: 100 batches of 32
Length of ValDataloader: 29 batches of 32
Length of TestDataloader: 15 batches of 32


  0%|          | 0/50 [00:00<?, ?it/s]

Train loss: 0.65957 | Train acc: 69.60%



  2%|▏         | 1/50 [00:04<03:54,  4.78s/it]

Val loss: 0.51507 | Val acc: 75.11%

Epoch 1 | LR: 3.000000e-04
Train loss: 0.43157 | Train acc: 81.39%



...

 98%|█████████▊| 49/50 [03:03<00:03,  3.75s/it]

Val loss: 0.11108 | Val acc: 97.47%

Epoch 49 | LR: 2.343750e-06
Train loss: 0.00653 | Train acc: 99.84%



100%|██████████| 50/50 [03:07<00:00,  3.75s/it]

Val loss: 0.11054 | Val acc: 97.58%

Epoch 50 | LR: 1.171875e-06


Test loss: 0.07680 | Test acc: 97.37%

Test: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2]
Predicted: [0 0 0 0 0 0 0 0 0